# Init

In [1]:
import os
import sys
import time
import boto3
import ncluster
import boto3
import base64
import pickle
import shlex
from ncluster import aws_util as u
from ncluster import util as util
import subprocess

from pprint import pprint as pprint
import importlib

sys.path.insert(0, os.environ['HOME'])
from credentials import *   # go_yaro, go_mind, go_andrew


def ossystem(cmd, shell=True):
    """Like os.system, but returns output of command as string."""
    p = subprocess.Popen(cmd, shell=shell, stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT)
    (stdout, stderr) = p.communicate()
    return stdout.decode('ascii')


ncluster version 0.1.73


In [ ]:
Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}])

In [ ]:
def easy_filter()

# Instances

## Launching EFA enabled instance

In [43]:
import importlib
client = u.get_ec2_client()
ec2 = u.get_ec2_resource()

In [4]:
  image = u.lookup_image('dlami23-efa')
  keypair = u.get_keypair()
  security_group = u.get_security_group()

In [61]:
os.environ['NCLUSTER_ZONE']='us-east-1d'

In [62]:
subnet=u.get_subnet()
placement_group = ''
name='efa-programmatic'

In [18]:
instance_type='p3dn.24xlarge'
disk_size=500

In [33]:
image.id

'ami-03147512dd1ffee88'

In [35]:
keypair.name

'ncluster-yaroslavvb'

In [54]:
security_group.id

'sg-01494dff7a2a16363'

In [63]:
subnet.id

'subnet-047707529f095f614'

In [55]:
image.id

'ami-03147512dd1ffee88'

In [56]:
keypair.name

'ncluster-yaroslavvb'

In [71]:
type(security_group)

boto3.resources.factory.ec2.SecurityGroup

In [72]:
import boto3

In [66]:
keypair_name='ncluster-yaroslavvb'
instance_type='p3dn.24xlarge'
instance_type='c5n.18xlarge'
security_group_id='sg-0139018179276c4ba'
subnet_id='subnet-047707529f095f614'
image_id='ami-03147512dd1ffee88'
zone='us-east-1d'

args = {'ImageId': image_id,
        'InstanceType': instance_type,
        'MinCount': 1,
        'MaxCount': 1,
        'KeyName': keypair_name}

args['TagSpecifications'] = [{
  'ResourceType': 'instance',
  'Tags': [{
    'Key': 'Name',
    'Value': name
  }]
}]

subnet = u.get_subnet()
args['NetworkInterfaces'] = [{'SubnetId': subnet_id,
                              'DeviceIndex': 0,
                              'DeleteOnTermination': True,
                              'InterfaceType':'efa',
                              'Groups': [security_group_id]}]
placement_specs = {'AvailabilityZone': zone}

args['Placement'] = placement_specs
args['Monitoring'] = {'Enabled': True}

In [67]:
instances = ec2.create_instances(**args)

In [68]:
boto3.__version__

'1.9.122'

In [25]:
!pip install -U boto3

    100% |████████████████████████████████| 133kB 240kB/s ta 0:00:01
    100% |████████████████████████████████| 5.5MB 1.3MB/s ta 0:00:011
  Found existing installation: botocore 1.12.122
    Uninstalling botocore-1.12.122:
      Successfully uninstalled botocore-1.12.122
  Found existing installation: boto3 1.9.122
    Uninstalling boto3-1.9.122:
      Successfully uninstalled boto3-1.9.122


## Launching EFA2

In [45]:
import boto3
instance_type='p3dn.24xlarge'

subnet_id='subnet-07e59d5b61d434fd4'
name='efa-programmatic'
image_id='ami-03147512dd1ffee88'
security_group_id='sg-01494dff7a2a16363'

args = {'ImageId': image_id,
        'InstanceType': instance_type,
        'MinCount': 1,
        'MaxCount': 1,
        'SecurityGroupIds': [security_group_id],
        'KeyName': 'ncluster-yaroslavvb'}

args['NetworkInterfaces'] = [{'SubnetId': subnet_id,
                               'DeviceIndex': 0,
                               'DeleteOnTermination': True,
                               'InterfaceType':'efa',
                               'Groups': [security_group_id]}]
placement_specs = {'AvailabilityZone': 'us-east-1b'}

ec2 = boto3.Session().resource('ec2')
instances = ec2.create_instances(**args)

ClientError: An error occurred (InvalidParameterCombination) when calling the RunInstances operation: Network interfaces and an instance-level security groups may not be specified on the same request

## Checking disks

In [3]:
instance=u.lookup_instance('demo')

Using region  us-east-1


In [33]:
volumes = list(instance.volumes.all())

In [35]:
volume = volumes[0]

In [36]:
volume.attachments

[{'AttachTime': datetime.datetime(2019, 6, 23, 22, 34, 54, tzinfo=tzutc()),
  'Device': '/dev/sda1',
  'InstanceId': 'i-048f463c7019b821e',
  'State': 'attached',
  'VolumeId': 'vol-00af8a22453ab79f0',
  'DeleteOnTermination': True}]

In [42]:
volume.size

500

In [43]:
volume.id

'vol-00af8a22453ab79f0'

In [37]:
volume.volume_type

'gp2'

In [38]:
volume.snapshots

ec2.Volume.snapshotsCollectionManager(ec2.Volume(id='vol-00af8a22453ab79f0'), ec2.Snapshot)

In [39]:
volume.attachments[0]['Device']

'/dev/sda1'

In [50]:
instance = u.lookup_instances('efa-image11')
for volume in instance.volumes.all():
    device = volume.attachments[0]['Device']
    print(f"{device:>10s} {volume.size:>10d} {volume.volume_type:>10s} {volume.id:>30s} ")

 /dev/sda1        500        gp2          vol-00af8a22453ab79f0 
 /dev/xvda        500        gp2          vol-063f8a82cc1ebc300 


## Checking security group

In [7]:
instance=u.lookup_instance('pair')

Using region  us-east-1


AssertionError: Didn't find any instances matching 'pair'

In [9]:
instance.placement_group.name

''

## Filtering instances by state

In [69]:
[u.get_name(i) for i in ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])]

['MindscribeWeb-env',
 'launch_interact-gpt1-ryanmckinney',
 'voice-server-auto',
 'voice-server-dev-server',
 'voice-server-auto']

# Network

## Getting default VPC

In [84]:
from ncluster import aws_util as u

In [85]:
u.get_vpc()

ec2.Vpc(id='vpc-0a007bc3954ed0c9d')

In [86]:
client = u.get_ec2_client()

In [87]:
client.describe_vpcs()

{'Vpcs': [{'CidrBlock': '172.31.0.0/16',
   'DhcpOptionsId': 'dopt-086557b289697d4b4',
   'State': 'available',
   'VpcId': 'vpc-d5c91aaf',
   'OwnerId': '331439827203',
   'InstanceTenancy': 'default',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-f6a70b9a',
     'CidrBlock': '172.31.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'IsDefault': True},
  {'CidrBlock': '192.168.0.0/16',
   'DhcpOptionsId': 'dopt-086557b289697d4b4',
   'State': 'available',
   'VpcId': 'vpc-0a007bc3954ed0c9d',
   'OwnerId': '331439827203',
   'InstanceTenancy': 'default',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-0a30b722e61658fc8',
     'CidrBlock': '192.168.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'IsDefault': False,
   'Tags': [{'Key': 'Name', 'Value': 'ncluster'}]}],
 'ResponseMetadata': {'RequestId': 'c0cb05a1-8ee4-4e30-a137-4f074ccbd48a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
 

In [88]:
u.get_default_vpc()

ec2.Vpc(id='vpc-d5c91aaf')

## Create EFA-friendly security group

In [ ]:
# 1. check existing security group that works
# 2. create new security group, double check that it looks the same

In [83]:
os.environ['AWS_DEFAULT_REGION']='us-east-1'
importlib.reload(boto3)
d = u.get_security_group_dict()
good_group=d['ncluster']

In [84]:
group.ip_permissions

[{'FromPort': 6006,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 6016,
  'UserIdGroupPairs': []},
 {'FromPort': 0,
  'IpProtocol': 'tcp',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 65535,
  'UserIdGroupPairs': [{'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'},
   {'GroupId': 'sg-09a65a86d87d54560',
    'PeeringStatus': 'active',
    'UserId': '331439827203',
    'VpcId': 'vpc-0a007bc3954ed0c9d',
    'VpcPeeringConnectionId': 'pcx-087644c72415b62e1'}]},
 {'FromPort': 8888,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 8899,
  'UserIdGroupPairs': []},
 {'IpProtocol': '-1',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'}]},
 {'FromPort': 60000,
  'IpProtocol': 'udp',
  'Ip

In [85]:
os.environ['AWS_DEFAULT_REGION']='us-west-2'
importlib.reload(boto3)
d = u.get_security_group_dict()
bad_group=d['ncluster']

In [86]:
good_group.id

'sg-01494dff7a2a16363'

In [87]:
bad_group.id

'sg-0481f4230bc91ab83'

In [90]:
good_group.id

'sg-01494dff7a2a16363'

In [88]:
good_group.ip_permissions

[{'FromPort': 6006,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 6016,
  'UserIdGroupPairs': []},
 {'FromPort': 0,
  'IpProtocol': 'tcp',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 65535,
  'UserIdGroupPairs': [{'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'},
   {'GroupId': 'sg-09a65a86d87d54560',
    'PeeringStatus': 'active',
    'UserId': '331439827203',
    'VpcId': 'vpc-0a007bc3954ed0c9d',
    'VpcPeeringConnectionId': 'pcx-087644c72415b62e1'}]},
 {'FromPort': 8888,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 8899,
  'UserIdGroupPairs': []},
 {'IpProtocol': '-1',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'}]},
 {'FromPort': 60000,
  'IpProtocol': 'udp',
  'Ip

### What correct rule looks like

In [ ]:
{'IpProtocol': '-1',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'}]},

In [73]:
group.id

'sg-069bb8d7ea3950cfc'

In [13]:
!aws ec2 create-security-group --group-name MySecurityGroup --description "My security group"

{
    "GroupId": "sg-0d9123fbedb51266f"
}


In [92]:
good_group.ip_permissions_egress

[{'IpProtocol': '-1',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'}]}]

In [ ]:
{'IpProtocol': '-1',
  'IpRanges': [],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-01494dff7a2a16363',
    'UserId': '331439827203'}]},

In [93]:
bad_group.ip_permissions_egress

[{'IpProtocol': '-1',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': []}]

In [109]:
bad_group.reload()
bad_group.ip_permissions_egress

[{'IpProtocol': '-1',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'UserIdGroupPairs': [{'Description': 'efa',
    'GroupId': 'sg-0481f4230bc91ab83',
    'UserId': '331439827203'}]}]

In [94]:
other_security_group=bad_group

In [107]:
rule = {
"IpProtocol": "-1",
"PrefixListIds": [],
'UserIdGroupPairs': [{'Description': 'efa',
                      'GroupId': other_security_group.id,
                      'UserId': u.get_account_number()}]
}
bad_group.authorize_egress(IpPermissions=[rule])

In [108]:
bad_group.authorize_egress(IpPermissions=[rule])

{'ResponseMetadata': {'RequestId': 'c537d59e-f7ca-432b-b6f7-434e0eef6715',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'content-length': '257',
   'date': 'Fri, 07 Jun 2019 22:34:25 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [14]:
!aws ec2 authorize-security-group-ingress --group-id sg-0d9123fbedb51266f --protocol all --port all --cidr 0.0.0.0/0

In [15]:
!aws ec2 describe-security-groups --group-names MySecurityGroup

{
    "SecurityGroups": [
        {
            "Description": "My security group",
            "GroupName": "MySecurityGroup",
            "IpPermissions": [
                {
                    "IpProtocol": "-1",
                    "IpRanges": [
                        {
                            "CidrIp": "0.0.0.0/0"
                        }
                    ],
                    "Ipv6Ranges": [],
                    "PrefixListIds": [],
                    "UserIdGroupPairs": []
                }
            ],
            "OwnerId": "331439827203",
            "GroupId": "sg-0d9123fbedb51266f",
            "IpPermissionsEgress": [
                {
                    "IpProtocol": "-1",
                    "IpRanges": [
                        {
                            "CidrIp": "0.0.0.0/0"
                        }
                    ],
                    "Ipv6Ranges": [],
                    "PrefixListIds": [],
                    "UserIdGroupPairs": []
       

### Create one through boto3

In [17]:
vpc = u.get_default_vpc()

In [33]:
security_group_name='MySecurityGroup3'

In [19]:
ec2 = u.get_ec2_resource()

In [34]:
group2=ec2.create_security_group(GroupName=security_group_name,
                          Description=security_group_name,
                          VpcId=vpc.id)

In [35]:
group2.id

'sg-0f8536aa45d3ab3ee'

In [24]:
rule={'IpProtocol': '-1',
      'IpRanges': [],
      'Ipv6Ranges': [],
      'PrefixListIds': [],
      'UserIdGroupPairs': [{'Description': 'efa',
        'GroupId': group2.id,
        'UserId': u.get_account_number()}]
     },

In [36]:
rule2= {
                    "IpProtocol": "-1",
                    "IpRanges": [
                        {
                            "CidrIp": "0.0.0.0/0"
                        }
                    ],
                    "Ipv6Ranges": [],
                    "PrefixListIds": [],
                    'UserIdGroupPairs': [{'Description': 'efa',
        'GroupId': group2.id,
        'UserId': u.get_account_number()}]
                }

In [40]:
group.description

'ncluster'

In [42]:
vpc.is_default

True

In [26]:
boto3.__version__

'1.9.159'

In [ ]:
pip install

In [37]:
response = group2.authorize_ingress(IpPermissions=[rule2])

In [38]:
response

{'ResponseMetadata': {'RequestId': 'b99465f3-fc28-41c7-99b0-b0e73be00641',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Fri, 07 Jun 2019 00:52:03 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Account-level

## Limits

In [3]:
client = u.get_ec2_client()

In [4]:
client.describe_account_attributes()

{'AccountAttributes': [{'AttributeName': 'supported-platforms',
   'AttributeValues': [{'AttributeValue': 'VPC'}]},
  {'AttributeName': 'vpc-max-security-groups-per-interface',
   'AttributeValues': [{'AttributeValue': '5'}]},
  {'AttributeName': 'max-elastic-ips',
   'AttributeValues': [{'AttributeValue': '5'}]},
  {'AttributeName': 'max-instances',
   'AttributeValues': [{'AttributeValue': '300'}]},
  {'AttributeName': 'vpc-max-elastic-ips',
   'AttributeValues': [{'AttributeValue': '5'}]},
  {'AttributeName': 'default-vpc',
   'AttributeValues': [{'AttributeValue': 'vpc-d5c91aaf'}]}],
 'ResponseMetadata': {'RequestId': 'c576523f-f2e6-436f-ba84-f96edadac907',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1858',
   'date': 'Fri, 21 Jun 2019 15:46:24 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Account name

- https://docs.aws.amazon.com/IAM/latest/APIReference/API_ListAccountAliases.html 
- https://docs.aws.amazon.com/cli/latest/reference/iam/list-account-aliases.html 

In [5]:
go_mind()
importlib.reload(boto3)
iam_client = boto3.client('iam')
iam_client.list_account_aliases()
iam_client.list_account_aliases()['AccountAliases']

['keyapp']

In [10]:
import os

In [17]:
import getpass
getpass.getuser()

'yaroslavvb'

In [18]:
iam_client.get_user()['User']['UserName']

'yaroslavvb'

In [3]:
# yaro
importlib.reload(boto3)
go_yaro()
iam_client = boto3.client('iam')
iam_client.list_account_aliases()

{'AccountAliases': ['yacht'],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '8901298c-8a1d-11e9-a205-2d1e7619d282',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8901298c-8a1d-11e9-a205-2d1e7619d282',
   'content-type': 'text/xml',
   'content-length': '384',
   'date': 'Sat, 08 Jun 2019 18:45:06 GMT'},
  'RetryAttempts': 0}}

In [4]:
# andrew
go_andrew()
importlib.reload(boto3)
iam_client = boto3.client('iam')
iam_client.list_account_aliases()

{'AccountAliases': ['antics'],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '8e8fe37b-8a1d-11e9-831f-2d10654543da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8e8fe37b-8a1d-11e9-831f-2d10654543da',
   'content-type': 'text/xml',
   'content-length': '385',
   'date': 'Sat, 08 Jun 2019 18:45:15 GMT'},
  'RetryAttempts': 0}}

In [5]:
# mindscribe
go_mind()
importlib.reload(boto3)
sts_client = boto3.client('sts')
pprint(sts_client.get_caller_identity())

{'Account': '979440106638',
 'Arn': 'arn:aws:iam::979440106638:user/yaroslavvb',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '407',
                                      'content-type': 'text/xml',
                                      'date': 'Sat, 08 Jun 2019 18:45:44 GMT',
                                      'x-amzn-requestid': '9fc0fc18-8a1d-11e9-8f18-af3e36405676'},
                      'HTTPStatusCode': 200,
                      'RequestId': '9fc0fc18-8a1d-11e9-8f18-af3e36405676',
                      'RetryAttempts': 0},
 'UserId': 'AIDA6ICZN7SHO54UXNLS3'}


## Valid zones for p3dn

In [22]:
import boto3
importlib.reload(boto3)

def valid_zones(prefix, init_func, instanceType):
    product = 'Linux/UNIX (Amazon VPC)'
    result = {}
    print("Valid zones for ", prefix, instanceType)
    for region in boto3.client('ec2').describe_regions()['Regions']:
        importlib.reload(boto3)
        client = boto3.client('ec2', region_name=region['RegionName'])
        for zone in [z['ZoneName'] for z in client.describe_availability_zones()['AvailabilityZones'] if z['State'] == 'available']:
            try:
                price = client.describe_spot_price_history(InstanceTypes=[instanceType],
                                                           MaxResults=1,
                                                           ProductDescriptions=[product],
                                                           AvailabilityZone=zone)['SpotPriceHistory'][0]['SpotPrice']
                print("%s: %s" % (zone, price))
                result[zone] = price
            except IndexError: pass
    return result

print(valid_zones('yacht', go_yacht, 'p3dn.24xlarge'))

Valid zones for  yacht p3dn.24xlarge
eu-west-1a: 9.739100
eu-west-1b: 9.739100
ap-northeast-1b: 42.783000
us-east-1b: 9.363600
us-east-1d: 9.363600
us-west-2b: 9.363600
us-west-2c: 9.363600
{'eu-west-1a': '9.739100', 'eu-west-1b': '9.739100', 'ap-northeast-1b': '42.783000', 'us-east-1b': '9.363600', 'us-east-1d': '9.363600', 'us-west-2b': '9.363600', 'us-west-2c': '9.363600'}


In [34]:
# get valid zones for accounts
valid_zone_dict = {}
for (prefix, init) in [('yacht', go_yacht), ('antics', go_antics), ('keyapp', go_keyapp)]:
    importlib.reload(boto3)

    for instance_type in ['p3.16xlarge', 'p3dn.24xlarge']:
        valid_zone_dict[(prefix, instance_type)] = valid_zones(prefix, init, instance_type)
print(valid_zone_dict)

Valid zones for  yacht p3.16xlarge
eu-west-2a: 8.613600
eu-west-2b: 8.613600
eu-west-1a: 7.932000
eu-west-1b: 7.932000
eu-west-1c: 26.440000
ap-northeast-2a: 11.954400
ap-northeast-2c: 11.954400
ap-northeast-1a: 10.065600
ap-northeast-1c: 10.065600
ca-central-1b: 8.078400
ap-southeast-1a: 10.161600
ap-southeast-1b: 10.161600
ap-southeast-2b: 10.161600
eu-central-1a: 9.175200
eu-central-1b: 9.175200
us-east-1a: 24.480000
us-east-1b: 7.344000
us-east-1c: 7.344000
us-east-1d: 7.344000
us-east-1f: 7.344000
us-east-2a: 7.344000
us-east-2b: 7.344000
us-west-2a: 7.344000
us-west-2b: 7.344000
us-west-2c: 7.344000
Valid zones for  yacht p3dn.24xlarge
eu-west-1a: 9.739100
eu-west-1c: 9.739100
ap-northeast-1a: 12.834900
us-east-1b: 9.363600
us-east-1c: 9.363600
us-west-2b: 9.363600
us-west-2c: 9.363600
Valid zones for  antics p3.16xlarge
eu-west-2a: 8.613600
eu-west-2b: 8.613600
eu-west-1a: 7.932000
eu-west-1b: 7.932000
eu-west-1c: 26.440000
ap-northeast-2a: 11.954400
ap-northeast-2c: 11.954400
a

In [6]:
d = {('yacht', 'p3.16xlarge'): {'eu-west-2a': '8.613600', 'eu-west-2b': '8.613600', 'eu-west-1a': '26.440000', 'eu-west-1b': '7.932000', 'eu-west-1c': '7.932000', 'ap-northeast-2a': '11.954400', 'ap-northeast-2c': '11.954400', 'ap-northeast-1b': '10.065600', 'ap-northeast-1c': '10.065600', 'ca-central-1b': '8.078400', 'ap-southeast-1a': '10.161600', 'ap-southeast-1b': '10.161600', 'ap-southeast-2a': '10.161600', 'eu-central-1a': '9.175200', 'eu-central-1b': '9.175200', 'us-east-1a': '24.480000', 'us-east-1b': '7.344000', 'us-east-1d': '7.344000', 'us-east-1e': '7.344000', 'us-east-1f': '7.344000', 'us-east-2a': '7.471400', 'us-east-2b': '7.344000', 'us-west-2a': '7.344000', 'us-west-2b': '7.344000', 'us-west-2c': '7.344000'}, ('yacht', 'p3dn.24xlarge'): {'eu-west-1a': '9.739100', 'eu-west-1b': '9.739100', 'ap-northeast-1b': '12.834900', 'us-east-1b': '9.363600', 'us-east-1d': '9.363600', 'us-west-2b': '9.363600', 'us-west-2c': '9.363600'}, ('antics', 'p3.16xlarge'): {'eu-west-2a': '8.613600', 'eu-west-2b': '8.613600', 'eu-west-1a': '26.440000', 'eu-west-1b': '7.932000', 'eu-west-1c': '7.932000', 'ap-northeast-2a': '11.954400', 'ap-northeast-2c': '11.954400', 'ap-northeast-1b': '10.065600', 'ap-northeast-1c': '10.065600', 'ca-central-1b': '8.078400', 'ap-southeast-1a': '10.161600', 'ap-southeast-1b': '10.161600', 'ap-southeast-2a': '10.161600', 'eu-central-1a': '9.175200', 'eu-central-1b': '9.175200', 'us-east-1a': '24.480000', 'us-east-1b': '7.344000', 'us-east-1d': '7.344000', 'us-east-1e': '7.344000', 'us-east-1f': '7.344000', 'us-east-2a': '7.471400', 'us-east-2b': '7.344000', 'us-west-2a': '7.344000', 'us-west-2b': '7.344000', 'us-west-2c': '7.344000'}, ('antics', 'p3dn.24xlarge'): {'eu-west-1a': '9.739100', 'eu-west-1b': '9.739100', 'ap-northeast-1b': '12.834900', 'us-east-1b': '9.363600', 'us-east-1d': '9.363600', 'us-west-2b': '9.363600', 'us-west-2c': '9.363600'}, ('keyapp', 'p3.16xlarge'): {'eu-west-2a': '8.613600', 'eu-west-2b': '8.613600', 'eu-west-1a': '26.440000', 'eu-west-1b': '7.932000', 'eu-west-1c': '7.932000', 'ap-northeast-2a': '11.954400', 'ap-northeast-2c': '11.954400', 'ap-northeast-1b': '10.065600', 'ap-northeast-1c': '10.065600', 'ca-central-1b': '8.078400', 'ap-southeast-1a': '10.161600', 'ap-southeast-1b': '10.161600', 'ap-southeast-2a': '10.161600', 'eu-central-1a': '9.175200', 'eu-central-1b': '9.175200', 'us-east-1a': '24.480000', 'us-east-1b': '7.344000', 'us-east-1d': '7.344000', 'us-east-1e': '7.344000', 'us-east-1f': '7.344000', 'us-east-2a': '7.471400', 'us-east-2b': '7.344000', 'us-west-2a': '7.344000', 'us-west-2b': '7.344000', 'us-west-2c': '7.344000'}, ('keyapp', 'p3dn.24xlarge'): {'eu-west-1a': '9.739100', 'eu-west-1b': '9.739100', 'ap-northeast-1b': '12.834900', 'us-east-1b': '9.363600', 'us-east-1d': '9.363600', 'us-west-2b': '9.363600', 'us-west-2c': '9.363600'}}

In [7]:
d[('yacht', 'p3.16xlarge')]

{'eu-west-2a': '8.613600',
 'eu-west-2b': '8.613600',
 'eu-west-1a': '26.440000',
 'eu-west-1b': '7.932000',
 'eu-west-1c': '7.932000',
 'ap-northeast-2a': '11.954400',
 'ap-northeast-2c': '11.954400',
 'ap-northeast-1b': '10.065600',
 'ap-northeast-1c': '10.065600',
 'ca-central-1b': '8.078400',
 'ap-southeast-1a': '10.161600',
 'ap-southeast-1b': '10.161600',
 'ap-southeast-2a': '10.161600',
 'eu-central-1a': '9.175200',
 'eu-central-1b': '9.175200',
 'us-east-1a': '24.480000',
 'us-east-1b': '7.344000',
 'us-east-1d': '7.344000',
 'us-east-1e': '7.344000',
 'us-east-1f': '7.344000',
 'us-east-2a': '7.471400',
 'us-east-2b': '7.344000',
 'us-west-2a': '7.344000',
 'us-west-2b': '7.344000',
 'us-west-2c': '7.344000'}

In [8]:
d[('keyapp', 'p3dn.24xlarge')]

{'eu-west-1a': '9.739100',
 'eu-west-1b': '9.739100',
 'ap-northeast-1b': '12.834900',
 'us-east-1b': '9.363600',
 'us-east-1d': '9.363600',
 'us-west-2b': '9.363600',
 'us-west-2c': '9.363600'}

In [31]:
d[('antics', 'p3dn.24xlarge')]

{'eu-west-1a': '9.739100',
 'eu-west-1b': '9.739100',
 'ap-northeast-1b': '12.834900',
 'us-east-1b': '9.363600',
 'us-east-1d': '9.363600',
 'us-west-2b': '9.363600',
 'us-west-2c': '9.363600'}

In [32]:
d[('yacht', 'p3dn.24xlarge')]

{'eu-west-1a': '9.739100',
 'eu-west-1b': '9.739100',
 'ap-northeast-1b': '12.834900',
 'us-east-1b': '9.363600',
 'us-east-1d': '9.363600',
 'us-west-2b': '9.363600',
 'us-west-2c': '9.363600'}

# Images

In [80]:
images = list(ec2.images.all())
[im for im in images if im.id == 'ami-a9d09ed1'][0].name

IndexError: list index out of range

In [ ]:
amazon_ami='ami-0cc96feef8c6bbff3'
ec2 = u.get_ec2_resource()
images = list(ec2.images.all())
[im for im in images if im.id == amazon_ami][0].name


## Get root device name

In [81]:
images = list(ec2.images.filter(ImageIds=['ami-0cc96feef8c6bbff3']));
device_name = images[0].root_device_name

In [82]:
image = images[0]

In [90]:
image.description

'Amazon Linux 2 AMI 2.0.20190612 x86_64 HVM gp2'

In [83]:
image.root_device_name

'/dev/xvda'

In [87]:
client = u.get_ec2_client()

In [89]:
client.describe_images(ImageIds=['ami-0cc96feef8c6bbff3'])

{'Images': [{'Architecture': 'x86_64',
   'CreationDate': '2019-06-11T07:15:13.000Z',
   'ImageId': 'ami-0cc96feef8c6bbff3',
   'ImageLocation': 'amazon/amzn2-ami-hvm-2.0.20190612-x86_64-gp2',
   'ImageType': 'machine',
   'Public': True,
   'OwnerId': '137112412989',
   'State': 'available',
   'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
     'Ebs': {'DeleteOnTermination': True,
      'SnapshotId': 'snap-08f742c6ac4f80dda',
      'VolumeSize': 8,
      'VolumeType': 'gp2',
      'Encrypted': False}}],
   'Description': 'Amazon Linux 2 AMI 2.0.20190612 x86_64 HVM gp2',
   'EnaSupport': True,
   'Hypervisor': 'xen',
   'ImageOwnerAlias': 'amazon',
   'Name': 'amzn2-ami-hvm-2.0.20190612-x86_64-gp2',
   'RootDeviceName': '/dev/xvda',
   'RootDeviceType': 'ebs',
   'SriovNetSupport': 'simple',
   'VirtualizationType': 'hvm'}],
 'ResponseMetadata': {'RequestId': '5b4677b0-8357-4c84-bfca-4a6914874746',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8

In [64]:
image.root_device_type

'ebs'

# Volumes

In [2]:
ec2 = u.get_ec2_resource()

In [21]:
u.get_name(v)

'noname'

In [24]:
v.volume_type

'gp2'

In [ ]:
volumes = list(ec2.volumes.all())
for v in volumes:
  print(v.id, u.get_name(v), v)

In [27]:
    volume_names = set(u.get_name(v) for v in ec2.volumes.all())


In [28]:
volume_names

{'2prepare-image', 'imagenet18', 'noname'}

In [31]:
f"{5:02d}"

'05'

In [4]:
volumes = list(ec2.volumes.all())
for v in volumes:
  if v.attachments:
    machine = u.get_name(ec2.Instance(v.attachments[0]['InstanceId']))
  else:
    machine = '<unattached>'
  print(v.id, u.get_name(v), machine)

vol-0d0f0b89221e5e821 noname 0.one_small_machine_wiki_state
vol-0417b338eca4208dd 2prepare-image 2prepare-image
vol-0caa9c257d63d99bc noname 4prepare
vol-053292dec5fff7965 noname tensorboard-east
vol-0b3370c415f9dc7cd noname 0.one-imagenet
vol-0e7e0928ccc431214 noname shell
vol-0f2293283ba7a254e noname pytorch_aws
vol-021050b054b05c6a4 noname sshtest
vol-0fef8436b3b5c0e99 noname mnist
vol-0ff25f669b4da1a84 noname ncluster_launch
vol-07f0daec1852ab6f1 noname efs_test
vol-0bddd83e9f307b0c3 noname shell
vol-0cd4da6eabe52cebb noname shell
vol-09fe7efe1b43e8e6b noname mnist
vol-04191ce2971c80d14 noname 1.mnist_distributed
vol-096daf0d410040175 noname 0.mnist_distributed
vol-0b7341b3dbe9da443 noname 0.mnist_distributed2
vol-09e4b8c3b7b590f57 noname 1.mnist_distributed2
vol-083dd0297aab919ae noname 2test
vol-01dd504f31802f2ab noname test
vol-05acff587a1a279b8 noname test2
vol-045366b5c3a90e53c noname imagenet_prep
vol-02c4b1c5ebe4db908 noname imagenet-prep
vol-0fd20d716517c942d imagenet18 <un

In [ ]:
ec2.volumes.filter()

In [5]:
instance = u.lookup_instance('imagenet-prep')

Using region  us-east-1


In [6]:
instance.attach_volume(Device='/dev/xvdh')

In [19]:
ec2.create_volume?

In [17]:
v =volumes[0]

In [18]:
v.size

1000

# Keypairs

Delete security group

In [37]:
u.get_keypair_fn()

'/Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem'

In [27]:
u.get_keypair_fn()

'/Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem'

In [38]:
pairs = u.get_keypair_dict()

In [39]:
pairs

{'dnew': ec2.KeyPairInfo(name='dnew'),
 'ncluster-ben': ec2.KeyPairInfo(name='ncluster-ben'),
 'ncluster-darius': ec2.KeyPairInfo(name='ncluster-darius'),
 'ncluster-dlam': ec2.KeyPairInfo(name='ncluster-dlam'),
 'ncluster-fakeben': ec2.KeyPairInfo(name='ncluster-fakeben'),
 'ncluster-ubuntu': ec2.KeyPairInfo(name='ncluster-ubuntu'),
 'ncluster-ubuntu2': ec2.KeyPairInfo(name='ncluster-ubuntu2'),
 'ncluster-yaroslav': ec2.KeyPairInfo(name='ncluster-yaroslav'),
 'ncluster-yaroslavvb': ec2.KeyPairInfo(name='ncluster-yaroslavvb')}

In [28]:
u.get_keypair_name()

'ncluster-yaroslavvb'

In [32]:
pairs

{'224u-ec2': ec2.KeyPairInfo(name='224u-ec2'),
 'aws-eb': ec2.KeyPairInfo(name='aws-eb'),
 'dlam': ec2.KeyPairInfo(name='dlam'),
 'ec2-test': ec2.KeyPairInfo(name='ec2-test'),
 'ncluster-darius': ec2.KeyPairInfo(name='ncluster-darius'),
 'ncluster-ryanmckinney': ec2.KeyPairInfo(name='ncluster-ryanmckinney'),
 'ncluster-ubuntu': ec2.KeyPairInfo(name='ncluster-ubuntu')}

## Fingerprinting

In [55]:
pair = pairs['ncluster-yaroslavvb']
print(u.get_keypair_fn())
print(pair.key_fingerprint)

/Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem
50:b9:04:03:b9:79:53:1d:cd:d2:1f:82:7b:ef:20:63:cb:19:26:26


In [79]:
print(f'openssl pkcs8 -in {u.get_keypair_fn()} -inform PEM -outform DER -topk8 -nocrypt | openssl sha1 -c')

openssl pkcs8 -in /Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem -inform PEM -outform DER -topk8 -nocrypt | openssl sha1 -c


In [77]:
!'openssl pkcs8 -in /Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem -inform PEM -outform DER -topk8 -nocrypt | openssl sha1 -c'

/bin/sh: openssl pkcs8 -in /Users/yaroslavvb/.ncluster/ncluster-yaroslavvb-979440106638-us-east-1.pem -inform PEM -outform DER -topk8 -nocrypt | openssl sha1 -c: No such file or directory


In [56]:
ossystem(f'openssl pkcs8 -in {u.get_keypair_fn()} -inform PEM -outform DER -topk8 -nocrypt | openssl sha1 -c')

'(stdin)= f2:8c:2a:ea:a3:da:ca:b0:7e:a0:d5:54:d3:5a:86:19:82:1e:b1:40\n'

Not the same :(

## Deleting keypairs

In [23]:
key_name = 'ncluster-yaroslavvb'
keypair = pairs[key_name]
print(f"Deleting keypair {key_name}")
ec2 = u.get_ec2_resource()
instance_list = []
for instance in ec2.instances.all():
    if instance.key_name == key_name:
        instance_list.append(instance)
if instance_list:
    print("Warning, after deleting keypair, the following instances will be no longer accessible:")
    for i in instance_list:
        print(u.get_name(i), i.id)
    answer = input("Proceed? (y/N)")
else:
    answer = "y"
if answer.lower() == 'y':
    keypair.delete()

Deleting keypair ncluster-yaroslavvb
Warning, the following instances will be no longer accessible:
gpubox i-00a15a75a001b9fab
train_squad-large-yaroslavvb i-038ab3bf2c5feeafc
Proceed? (y/N)y


In [21]:
instance.id

'i-038ab3bf2c5feeafc'

# Scratch